# NLP Tokenizers - Character based, n-gram, BPE

In [1]:
from torchtext import data
from torchtext import datasets

C:\Users\user\example5\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
print(sys.version)

3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]


In [3]:
''' 사용한 패키지 버전
pip 21.2.4
torch 1.11.0+cu113
torchtext 0.6.0
torchdata 0.3.0
transformers 4.18.0
sentencepiece 0.1.96
'''

' 사용한 패키지 버전\npip 21.2.4\ntorch 1.11.0+cu113\ntorchtext 0.6.0\ntorchdata 0.3.0\n'

In [4]:
TEXT = data.Field(lower=True, batch_first = True)

In [5]:
LABEL = data.Field(sequential=False)

In [6]:
train, test = datasets.IMDB.splits(TEXT,LABEL)

# Corpus & Out of Vocabulary (OOV)

In [7]:
s1 = '나는 책상 위에 사과를 먹었다'
s2 = '알고 보니 그 사과는 Jason 것이었다'
s3 = '그래서 Jason에게 사과를 했다'

print(s1.split())

['나는', '책상', '위에', '사과를', '먹었다']


In [8]:
print(s2.split())

['알고', '보니', '그', '사과는', 'Jason', '것이었다']


In [9]:
print(s3.split())

['그래서', 'Jason에게', '사과를', '했다']


In [10]:
print(list(s1))

['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', '먹', '었', '다']


In [11]:
token2idx = {}
index = 0

for sentence in [s1,s2,s3]:
    tokens = sentence.split()
    for token in tokens:
        if token2idx.get(token) == None:
            token2idx[token] = index
            index += 1
            
print(token2idx)

{'나는': 0, '책상': 1, '위에': 2, '사과를': 3, '먹었다': 4, '알고': 5, '보니': 6, '그': 7, '사과는': 8, 'Jason': 9, '것이었다': 10, '그래서': 11, 'Jason에게': 12, '했다': 13}


In [12]:
def indexed_sentence(sentence):
    return [token2idx[token] for token in sentence]

s1_i = indexed_sentence(s1.split())
print(s1_i)

[0, 1, 2, 3, 4]


In [13]:
s2_i = indexed_sentence(s2.split())
print(s2_i)

[5, 6, 7, 8, 9, 10]


In [14]:
s3_i = indexed_sentence(s3.split())
print(s3_i)

[11, 12, 3, 13]


In [15]:
'''OOV (Out Of Vocabulary)'''
s4 = '나는 책상 위에 배를 먹었다'

In [16]:
token2idx = {t:i+1 for t, i in token2idx.items()}
token2idx['<unk>'] = 0

In [17]:
token2idx

{'나는': 1,
 '책상': 2,
 '위에': 3,
 '사과를': 4,
 '먹었다': 5,
 '알고': 6,
 '보니': 7,
 '그': 8,
 '사과는': 9,
 'Jason': 10,
 '것이었다': 11,
 '그래서': 12,
 'Jason에게': 13,
 '했다': 14,
 '<unk>': 0}

In [18]:
def indexed_sentence_unk(sentence):
    return [token2idx.get(token,token2idx['<unk>']) for token in sentence]

indexed_sentence_unk(s4.split())

[1, 2, 3, 0, 5]

## 1. Character based tokenization

In [19]:
print([chr(k) for k in range(65,91)])

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [20]:
print([chr(k) for k in range(97,123)])

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [21]:
print([chr(k) for k in range(32,48)])

[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/']


In [22]:
print([chr(k) for k in range(58,65)])

[':', ';', '<', '=', '>', '?', '@']


In [23]:
print([chr(k) for k in range(91,97)])

['[', '\\', ']', '^', '_', '`']


In [24]:
print([chr(k) for k in range(123,127)])

['{', '|', '}', '~']


In [25]:
print([chr(k) for k in range(48,58)])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [26]:
print([chr(k) for k in range(int('0xAC00',16), int('0xD7A3',16)+1)][:50])

['가', '각', '갂', '갃', '간', '갅', '갆', '갇', '갈', '갉', '갊', '갋', '갌', '갍', '갎', '갏', '감', '갑', '값', '갓', '갔', '강', '갖', '갗', '갘', '같', '갚', '갛', '개', '객', '갞', '갟', '갠', '갡', '갢', '갣', '갤', '갥', '갦', '갧', '갨', '갩', '갪', '갫', '갬', '갭', '갮', '갯', '갰', '갱']


In [27]:
print([chr(k) for k in range(int('0xAC00',16), int('0xD7A3',16)+1)][4000:4050])

['뮠', '뮡', '뮢', '뮣', '뮤', '뮥', '뮦', '뮧', '뮨', '뮩', '뮪', '뮫', '뮬', '뮭', '뮮', '뮯', '뮰', '뮱', '뮲', '뮳', '뮴', '뮵', '뮶', '뮷', '뮸', '뮹', '뮺', '뮻', '뮼', '뮽', '뮾', '뮿', '므', '믁', '믂', '믃', '믄', '믅', '믆', '믇', '믈', '믉', '믊', '믋', '믌', '믍', '믎', '믏', '믐', '믑']


In [28]:
print([chr(k) for k in range(int('0x3131',16),int('0x3163',16)+1)])

['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']


In [29]:
idx2char = {0:'<pad>', 1:'<unk>'}

str_idx = len(idx2char)

for x in range(32,127):
    idx2char.update({str_idx: chr(x)})
    str_idx +=1
    
for x in range(int('0x3131',16),int('0x3163',16)+1):
    idx2char.update({str_idx:chr(x)})
    str_idx += 1
    
for x in range(int('0xAC00',16),int('0xD7A3',16)+1):
    idx2char.update({str_idx:chr(x)})
    str_idx += 1
    

In [30]:
char2idx = {v:k for k, v in idx2char.items()}

In [31]:
print([char2idx.get(c,0) for c in '그래서 Jason에게 사과를 했다']) #띄어쓰기가 2

[652, 3116, 5552, 2, 44, 67, 85, 81, 80, 6756, 288, 2, 5440, 400, 3600, 2, 10780, 1912]


In [32]:
print([char2idx.get(c,0) for c in 'ㅇㅋ! ㄳㄳ']) 

[119, 123, 3, 2, 99, 99]


In [33]:
print([char2idx.get(c,0) for c in 'ㅇㅋ! ㄱㅅㄱㅅ']) 

[119, 123, 3, 2, 97, 117, 97, 117]


## 2. n-gram tokenization

In [34]:
s1 = '나는 책상 위에 사과를 먹었다'

print([s1[i:i+1] for i in range(len(s1))])

['나', '는', ' ', '책', '상', ' ', '위', '에', ' ', '사', '과', '를', ' ', '먹', '었', '다']


In [35]:
print([s1[i:i+2] for i in range(len(s1))])

['나는', '는 ', ' 책', '책상', '상 ', ' 위', '위에', '에 ', ' 사', '사과', '과를', '를 ', ' 먹', '먹었', '었다', '다']


In [36]:
print([s1[i:i+3] for i in range(len(s1))])

['나는 ', '는 책', ' 책상', '책상 ', '상 위', ' 위에', '위에 ', '에 사', ' 사과', '사과를', '과를 ', '를 먹', ' 먹었', '먹었다', '었다', '다']


In [37]:
#띄어쓰기 기준에 적용

s2 = 'I am dying to play the game'

print([s2.split()[i:i+3] for i in range(len(s2.split()))])

[['I', 'am', 'dying'], ['am', 'dying', 'to'], ['dying', 'to', 'play'], ['to', 'play', 'the'], ['play', 'the', 'game'], ['the', 'game'], ['game']]


In [38]:
#띄어쓰기 기준에 적용

s3 = '너 때문에 간 떨어질 뻔했다'

print([s3.split()[i:i+3] for i in range(len(s3.split()))])

[['너', '때문에', '간'], ['때문에', '간', '떨어질'], ['간', '떨어질', '뻔했다'], ['떨어질', '뻔했다'], ['뻔했다']]


## 3. BPE(Byte Pair Encoding)

In [39]:
'''Algorithm : Learn BEP operations'''

import re, collections
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs
            
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\\s)'+bigram+r'(?!\\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair),word)
        v_out[w_out] = v_in[word]
    return v_out

In [40]:
vocab = {'l o w </w>':5, 'l o w e r </w>' : 2,
        'n e w e s t </w>':6, 'w i d e s t </w>':3}
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(f'Step {i+1}')
    print(best)
    print(vocab)
    print('\\n')

Step 1
('e', 's')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}
\n
Step 2
('es', 't')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}
\n
Step 3
('est', '</w>')
{'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}
\n
Step 4
('l', 'o')
{'lo w </w>': 5, 'lo w e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}
\n
Step 5
('lo', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'n e w est</w>': 6, 'w i d est</w>': 3}
\n
Step 6
('n', 'e')
{'low </w>': 5, 'low e r </w>': 2, 'ne w est</w>': 6, 'w i d est</w>': 3}
\n
Step 7
('ne', 'w')
{'low </w>': 5, 'low e r </w>': 2, 'new est</w>': 6, 'w i d est</w>': 3}
\n
Step 8
('new', 'est</w>')
{'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}
\n
Step 9
('low', '</w>')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}
\n
Step 10
('w', 'i')
{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}
\n


In [41]:
s1 = '나는 책상 위에 사과를 먹었다'
s2 = '알고 보니 그 사과는 Jason 것이었다'
s3 = '그래서 Jason에게 사과를 했다'

In [42]:
token_counts = {}
index = 0

for sentence in [s1,s2,s3]:
    tokens = sentence.split()
    for token in tokens:
        if token_counts.get(token) == None:
            token_counts[token] = 1
        else:
            token_counts[token] +=1
            
token_counts = {" ".join(token):counts for token, counts in token_counts.items()}
print(token_counts)

{'나 는': 1, '책 상': 1, '위 에': 1, '사 과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사 과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}


In [43]:
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(token_counts)
    best = max(pairs, key=pairs.get)
    token_counts = merge_vocab(best, token_counts)
    print(f'Step {i+1}')
    print(best)
    print(token_counts)
    print('\\n')

Step 1
('사', '과')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과 를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}
\n
Step 2
('사과', '를')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었 다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었 다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}
\n
Step 3
('었', '다')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'J a s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'J a s o n 에 게': 1, '했 다': 1}
\n
Step 4
('J', 'a')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Ja s o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Ja s o n 에 게': 1, '했 다': 1}
\n
Step 5
('Ja', 's')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보 니': 1, '그': 1, '사과 는': 1, 'Jas o n': 1, '것 이 었다': 1, '그 래 서': 1, 'Jas o n 에 게': 1, '했 다': 1}
\n
Step 6
('Jas', 'o')
{'나 는': 1, '책 상': 1, '위 에': 1, '사과를': 2, '먹 었다': 1, '알 고': 1, '보

# Pre-Trained Tokenizer 사용하기

In [52]:
import sentencepiece as spm
#s = spm.SentencePieceProcessor(model_file = 'spm.model')
#for n in range(5):
#    s.encode('New York', out_type=str, enable_sampling = True, alpha = 0.1, nbest = -1)

In [53]:
from transformers import BertTokenizer

In [51]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(len(tokenizer.vocab))

Downloading: 100%|███████████████████████████████████████████████████████████████████| 226k/226k [00:00<00:00, 286kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 28.0/28.0 [00:00<00:00, 5.59kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 145kB/s]

30522


In [54]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence = 'My dog is cute. He likes playing'
print(tokenizer.tokenize(sentence))

['my', 'dog', 'is', 'cute', '.', 'he', 'likes', 'playing']


In [55]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
print(len(tokenizer.vocab))

Downloading: 100%|███████████████████████████████████████████████████████████████████| 851k/851k [00:01<00:00, 586kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 28.0/28.0 [00:00<00:00, 5.61kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 625/625 [00:00<00:00, 157kB/s]

105879


In [56]:
print(tokenizer.tokenize(sentence))

['my', 'dog', 'is', 'cut', '##e', '.', 'he', 'likes', 'playing']


In [57]:
sentence = '나는 책상 위에 사과를 먹었다. 알고 보니 그 사과는 Jason 것이었다. 그래서 Jason에게 사과를 했다'

In [58]:
print(tokenizer.tokenize(sentence))

['나는', 'ᄎ', '##ᅢᆨ', '##상', '위에', 'ᄉ', '##ᅡ', '##과', '##를', 'ᄆ', '##ᅥ', '##ᆨ', '##었다', '.', '알', '##고', 'ᄇ', '##ᅩ', '##니', '그', 'ᄉ', '##ᅡ', '##과', '##는', 'jason', '것이', '##었다', '.', '그', '##래', '##서', 'jason', '##에게', 'ᄉ', '##ᅡ', '##과', '##를', '했다']
